## GPTQ with `meta-llama/Meta-Llama-3-8B-Instruct`

In this tutorial, we will demonstrate how to apply GPTQ to Llama-3-8B instruct to quantize the weights to 4 bits.

## Install

Get started by installing SparseML via pip. You will need a GPU instance.

In [ ]:
!pip install sparseml[transformers]==1.8

### 1) Load Model

First, load a model from the Hugging Face hub (in this case `Meta-Llama-3-8B-Instruct`) using `SparseAutoModelForCausalLM`.

* `SparseAutoModelForCausalLM` is a wrapper around `AutoModelForCausalLM`, with some added utilities for saving and loading quantized models.

In [ ]:
import torch
from sparseml.transformers import SparseAutoModelForCausalLM
from transformers import AutoTokenizer

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

model = SparseAutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

### 2) Dataset

Next, load a dataset for calibrating the model. 

Best practices for calibration data:
* Apply the model's chat template to the sample data
* Use at least 512 samples. 1024 samples can improve the results sometimes
* Use at least 2048 sequence length. 4096 can improve the results sometimes
* Select a diverse, high quality dataset (ideally that is adapted to your use case)

In this case, we will use the [`HuggingFaceH4/ultrachat_200k` dataset](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k), which contains multi-turn conversations and is generally a good choice for chat models.

In [ ]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES = 512

ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
ds = ds.shuffle(seed=42).select(range(NUM_CALIBRATION_SAMPLES))

# Dataset should have "text" field with the data you want to use to calibrate
ds = ds.map(lambda batch: {
    "text": tokenizer.apply_chat_template(batch["messages"], tokenize=False)
})

### 3) Recipe

Next, we make a recipe to specify the quantization algorithm to apply. In this case, we will use the `GPTQModifier`, which quantizes the weights of the model using GPTQ.

We will target all linear layers except the lm-head with:
- 4 bits 
- symmetric quantization
- groups of 128

This scheme is generally a good choice for making accurate models.

In [ ]:
recipe = """
quant_stage:
    quant_modifiers:
        GPTQModifier:
            sequential_update: false
            ignore: ["lm_head"]
            config_groups:
                group_0:
                    weights:
                        num_bits: 4
                        type: "int"
                        symmetric: true
                        strategy: "group"
                        group_size: 128
                    targets: ["Linear"]
"""

### 4) Apply The Algorithm

After making the recipe, we can apply the quantization algorithm using the `oneshot` function.

> WARNING: You will need about 60GB of GPU RAM to run the below. To reduce memory consumption at the expense of speed, set `sequential_update: true` in your recipe.

In [ ]:
from sparseml.transformers import oneshot

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=2048,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

In [ ]:
# Confirm generations of the quantized model look sane
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))

### 5) Serialize the model

Save the model using `save_pretrained` using `save_compressed=True`. This will save the weights in a compressed format, compatible for loading with vLLM!

In [ ]:
OUTPUT_DIR = "llama-3-gptq-4-bit"
model.save_pretrained(OUTPUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUTPUT_DIR)